In [ ]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# chronocluster
from chronocluster.data.dataio import pts_df_to_gis, kde_to_geotiff
from chronocluster import clustering
from chronocluster.utils import clustering_heatmap, pdiff_heatmap, get_box, chrono_plot, draw_ellipses
from chronocluster.distributions import ddelta
from chronocluster.density import kde_time, custom_kde, pymc_gmm_peak_finder, kde_peaks, rank_peaks

In [ ]:
# data wrangling
df = pd.read_csv('../Data/temples.csv')
df = df.dropna(subset=['xeast', 'ynorth', 'date'])
df

In [ ]:
points = [
    clustering.Point(
        x=row['xeast'],
        y=row['ynorth'],
        start_distribution = ddelta(row['date']),
        end_distribution = ddelta(1500)
    )
    for _, row in df.iterrows()
]

points

In [ ]:
# Custom styling parameters
style_params = {
    'start_mean_color': None,  # Do not plot start mean points
    'end_mean_color': None, # Do not plot end mean points
    'mean_point_size': 10,
    'cylinder_color': (0.3, 0.3, 0.3),  # Dark grey
    'ppf_limits': (0.05, 0.95),  # Use different ppf limits
    'shadow_color': (0.4, 0.4, 0.4),  # grey
    'shadow_size': 10,
    'time_slice_color': (0.5, 0.5, 0.5),  # Grey
    'time_slice_alpha': 0.3,
    'time_slice_point_color': (0, 0, 0),  # Black
}

# Plot the points using the chrono_plot function with custom styling and a time slice plane
ax = chrono_plot(points, style_params=style_params, time_slice=900)
ax.set_box_aspect(None, zoom=0.85)

In [ ]:
# Define the time slices
start_time = 800
end_time = 1200
time_interval = 50
time_slices = np.arange(start_time, end_time, time_interval)

In [ ]:
# Run the Monte Carlo simulation to get an ensemble of probable 
# lists of points included in each time slice.
num_iterations = 100 # sets the number of draws for incorporating chronological uncertainty
simulations = clustering.mc_samples(points, 
                                    time_slices=time_slices,  
                                    num_iterations=num_iterations)

# Get a bounding box for use later and to extract sensible distance limits
x_min, y_min, x_max, y_max = get_box(points)
max_distance = np.ceil(np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2))

# set consistent pairwise bandwidth (binning of distances)
use_kde = True
pair_bw = 100
kde_sample_n = 20
if use_kde:
    pair_bw = None#0.5 * pair_bw

# Produce pairwise distances to explore clustering structure
pairwise_density, support = clustering.temporal_pairwise(simulations, 
                                                         time_slices, 
                                                         bw=pair_bw, 
                                                         use_kde=use_kde, 
                                                         kde_sample_n=kde_sample_n,
                                                         max_distance=max_distance)

print(pairwise_density.shape)

# Visualize clustering with heatmap
clustering_heatmap(pairwise_density,
                   support,
                   time_slices,
                   result_type='Pairwise Distances')


In [ ]:
# Get MC iterations for incorporating chronological uncertainty and CSR
csr_simulations = clustering.mc_samples(points, 
                                        time_slices = time_slices,  
                                        num_iterations = num_iterations,
                                        null_model=clustering.csr_sample,
                                        x_min=x_min, 
                                        x_max=x_max,
                                        y_min=y_min, 
                                        y_max=y_max)

# Calulate the pairwise distances for the CSR sample
csr_pairwise_density, csr_support = clustering.temporal_pairwise(csr_simulations, 
                                                                 time_slices, 
                                                                 bw = pair_bw, 
                                                                 use_kde = use_kde,
                                                                 kde_sample_n=kde_sample_n, 
                                                                 max_distance = max_distance)

# Visualize clustering with heatmap
clustering_heatmap(csr_pairwise_density,
                   csr_support,
                   time_slices,
                   result_type='Pairwise Distances')


In [ ]:
# Calculate the p-values for density differences between the observed points and 
# the simulated CSR baseline per distance and temporal slice
p_diff_array, diff_array = clustering.p_diff(pairwise_density, csr_pairwise_density)

# Plot the heatmap of probabilities
pdiff_heatmap(p_diff_array,
              time_slices,
              csr_support)

In [ ]:
# Get MC iterations for incorporating chronological uncertainty with BISE
bise_simulations = clustering.mc_samples(points, 
                                         time_slices, 
                                         num_iterations=num_iterations,
                                         null_model=clustering.bise)

# Calulate the pairwise distances for the LISE sample
bise_pairwise_density, bise_support = clustering.temporal_pairwise(bise_simulations, 
                                                                 time_slices, 
                                                                 bw = pair_bw, 
                                                                 use_kde = use_kde,
                                                                 kde_sample_n=kde_sample_n, 
                                                                 max_distance = max_distance)

# Calculate the p-values for density differences between the observed points and 
# the simulated CSR baseline per distance and temporal slice
p_diff_array, diff_array = clustering.p_diff(pairwise_density, bise_pairwise_density)

# Plot the heatmap of probabilities
pdiff_heatmap(p_diff_array,
              time_slices,
              bise_support)

In [ ]:
from chronocluster.utils import plot_pdd

time_slice_idx = np.where(time_slices == 900)[0][0]  # corresponding to time 1100

# List of density arrays
density_arrays = [pairwise_density, csr_pairwise_density, bise_pairwise_density]

# Generate the plot and get the figure and axis objects
fig, ax = plot_pdd(
    time_slices=time_slices,
    time_slice_idx=time_slice_idx,
    support=support,
    density_arrays=density_arrays,
    quantiles=[0.025, 0.975],
    density_names=["Empirical", "CSR", "BISE"],
    colors=["blue", "orange", "green"]
)

# Show the plot
plt.show()

In [ ]:
# List of density arrays
density_arrays = [diff_array]

# Generate the plot and get the figure and axis objects
fig, ax = plot_pdd(
    time_slices=time_slices,
    time_slice_idx=time_slice_idx,
    support=support,
    density_arrays=density_arrays,
    quantiles=[0.025, 0.975],
    density_names=["Diff Array"],
    colors=["blue"]
)

# Add a horizontal line at y=0
ax.axhline(y=0, color='red', linestyle='--', linewidth=1.5)

# Show the plot
plt.show()

In [ ]:
time_slices

In [ ]:
# List of time_slice_idx values
time_slice_indices = [0, 2, 4, 6]  # Replace with your actual indices

# Create a figure and axes for subplots
num_panels = len(time_slice_indices)
fig, axes = plt.subplots(1, num_panels, figsize=(5 * num_panels, 5), sharey=True)  # 1 row, multiple columns

# Loop through each time_slice_idx and generate the plots
for idx, (ax, time_slice_idx) in enumerate(zip(axes, time_slice_indices)):
    # Generate the plot for the current time_slice_idx
    fig, _ = plot_pdd(
        time_slices=time_slices,
        time_slice_idx=time_slice_idx,
        support=support,
        density_arrays=density_arrays,
        quantiles=[0.025, 0.975],
        density_names=["Diff Array"],
        colors=["blue"],
        ax=ax
    )
    
    # Add a horizontal line (optional)
    ax.axhline(y=0.5, color='red', linestyle='--', linewidth=1.5)
    
    # Add a title for each panel
    ax.set_title(f"Time Slice {time_slice_idx}")

# Adjust layout and show the stitched plot
plt.tight_layout()
plt.show()

In [ ]:
# identify and save one or more characteristic scales
characteristic_scales = [5000]

# Define grid resolution and create the 2D grid for KDE evaluation
# Get a bounding box for use later and to extract sensible distance limits
x_min, y_min, x_max, y_max = get_box(points)
max_distance = np.ceil(np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2))

grid_resolution = 100  # Adjust the number of points as needed for resolution
x_grid = np.linspace(x_min, x_max, grid_resolution)
y_grid = np.linspace(y_min, y_max, grid_resolution)
x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)
grid = np.vstack([x_mesh.ravel(), y_mesh.ravel()]).T  # Flatten the grid for KDE input

In [ ]:
# Set parameters for KDE, including arbitrary time_slice from simulated data
time_slice = time_slices[time_slice_idx]
bandwidth =  characteristic_scales[0] * 0.5

# Calculate KDE for the chosen time slice
kde_values = kde_time(points, 
                      time_slice, 
                      bandwidth, 
                      grid, 
                      output_shape=x_mesh.shape, 
                      kde_method=custom_kde)

# Plotting
plt.contourf(x_mesh, y_mesh, kde_values, levels=20, cmap='viridis')
plt.colorbar(label="KDE Density")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title(f"KDE for Time Slice {time_slice}")
plt.show()

In [ ]:
kde_to_geotiff(x_mesh, 
               y_mesh, 
               kde_values, 
               epsg_code=32648, 
               output_path="../Output/angkor_temples_kde_output.tif")

In [ ]:
# Calculate the spatial extent based on bounding box to constrain the prior for
# the component means (parameter space outside this area is going to be fruitless)
bounding_box_variance = ( max_distance/ 2)**2

# Set maximum number of components to allow in the model
max_components = 8
w_threshold = 1 / max_components # used for idenitifying peak importance

# Priors for spatial scale (variance) based on pairwise distance density analysis
target_scale = bandwidth  # This is our target spatial scale for each component
target_scale_sd = 100  # Some variation around this value

# Run kde_peaks with GMM as the peak-finding method
# Assuming coordinates is your dataset of temple locations, passed as Point objects
peaks, weights, trace = kde_peaks(points=points, 
                                    num_peaks=max_components, 
                                    peak_finder=pymc_gmm_peak_finder,
                                    time_slice = time_slice,
                                    target_scale = target_scale,
                                    target_scale_sd = target_scale_sd,
                                    w_threshold = w_threshold,
                                    sampler = 'NUTS',
                                    draws = 2000,
                                    tune = 4000,
                                    chains = 1)

In [ ]:
importance_hdi = 0.80
summary_df = rank_peaks(trace, significance=importance_hdi, source_param='importance')

# isolate important peaks
# Filter rows where the lower bound of the HDI is greater than importance_threshold
importance_threshold = 0
condition = summary_df[f'{int(importance_hdi * 100)}% HDI (Importance)'].apply(lambda hdi: hdi[0] > importance_threshold)
important_peaks = summary_df[condition] # isloated for plotting below
summary_df

In [ ]:
pts_df_to_gis(summary_df,
              epsg_code=32648,
              output_path="../Output/angkor_temple_cluster_centres.gpkg", 
              file_format="GPKG")

In [ ]:
# Plotting
fig, ax = plt.subplots()

# Plot KDE density surface
ax.contourf(x_mesh, y_mesh, kde_values, levels=20, cmap='viridis')

# Extract the X and Y coordinates from the Coordinates column for plotting
x_coords = important_peaks['Coordinates'].apply(lambda coord: coord[0])
y_coords = important_peaks['Coordinates'].apply(lambda coord: coord[1])

# Plot the original data points
#ax.scatter(summary_df['x'], summary_df['y'], color='white', marker='o', s=5, label='Original Data')

# Draw ellipses for the GMM components with 1 SD and 2 SD ranges
draw_ellipses(ax, 
              important_peaks, 
              std_devs=[1, 2], 
              edgecolor='red', 
              facecolor='none', 
              linestyle='--', 
              linewidth=1)

# Annotate each component with its rank
for x, y, rank in zip(x_coords, y_coords, important_peaks['Rank']):
    ax.text(x, y, rank, color='black', ha='center', va='center', fontsize=12)

# Add legend, labels, and title
ax.legend()
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.set_title("KDE with GMM Peaks Ranked by Importance")

plt.show()